# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import os
import numpy as np
import pandas as pd

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
print('current dir: ' + os.getcwd())

current dir: /Users/sherzodn/Documents/Training/springBoard/Case Studies/Big Mountain Resort_GuidedCapstone/raw data


In [3]:
# change dir to data
path = '/Users/sherzodn/Documents/Training/springBoard/Case Studies/Big Mountain Resort_GuidedCapstone/data'
os.chdir(path)

In [4]:
df = pd.read_csv('step_3_output.csv', index_col = 0)

In [5]:
# check uploaded file/created df
df.head()

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,0,1,...,30.0,30.000000,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,2
1,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,2,3,...,800.0,80.000000,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,0
2,Yosemite Ski & Snowboard Area,California,7800,600,0,0.0,0,0,0,1,...,88.0,174.873239,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,0
3,Boreal Mountain Resort,California,7700,500,0,0.0,0,1,1,3,...,380.0,200.000000,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,0
4,Dodge Ridge,California,8200,1600,0,0.0,0,0,1,2,...,862.0,174.873239,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,0


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [6]:
# get dummies
#df = pd.get_dummies(df, prefix=['State'], columns=['state'])
df = pd.concat([df, pd.get_dummies(df['state'])], axis=1).drop(['state'], axis =1)
print(df.shape)
df.head()

(176, 60)


,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,Rhode Island,South Dakota,Tennessee,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Hilltop Ski Area,2090,294,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Sunrise Park Resort,11100,1800,0,0.0,0,1,2,3,1,...,0,0,0,0,0,0,0,0,0,0
2,Yosemite Ski & Snowboard Area,7800,600,0,0.0,0,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0
3,Boreal Mountain Resort,7700,500,0,0.0,0,1,1,3,1,...,0,0,0,0,0,0,0,0,0,0
4,Dodge Ridge,8200,1600,0,0.0,0,0,1,2,5,...,0,0,0,0,0,0,0,0,0,0


## df.head()

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [7]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing as prp

# Declare an explanatory variable, called x,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
x = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter x 
scaler = prp.StandardScaler().fit(x)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter x 
x_scaled=scaler.transform(x) 

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [8]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [9]:
# all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(x_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [10]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(x_test)

## Review Model Outcomes — Iterate over additional models as needed

In [11]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics.
from sklearn.metrics import explained_variance_score, mean_absolute_error

In [12]:
# You can plug y_test and y_pred into the functions to evaluate the model
evs = explained_variance_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [13]:
mae

75847653345.73671

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [14]:
lm.intercept_

-18961913278.56014

**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [15]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
#pd.DataFrame(abs(lm.coef_), x.columns, columns=['Coefficient'])
coeff_df = pd.DataFrame(abs(lm.coef_), x.columns, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
fastSixes,1.994939e+13
trams,1.689439e+13
fastEight,4.729028e+12
total_chairs,2.434161e+12
double,1.336320e+12
surface,1.278832e+12
triple,1.091526e+12
New York,1.059154e+12
Michigan,9.318382e+11
California,8.411907e+11


#### Model 2

**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

In [ ]:
# Removing states columns

In [35]:
states = ['Alaska', 'Arizona', 'California',
       'Colorado', 'Connecticut', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Missouri', 'Montana', 'Nevada', 'New Hampshire', 'New Jersey',
       'New Mexico', 'New York', 'North Carolina', 'Ohio', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Dakota', 'Tennessee', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin',
       'Wyoming']

In [37]:
df.drop(states, axis=1, inplace=True)

In [38]:
df.head()

,Name,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Hilltop Ski Area,2090,294,0,0.0,0,0,0,1,0,...,30.0,30.000000,150.000000,36.0,69.0,30.0,34.00000,152.0,30.0,2
1,Sunrise Park Resort,11100,1800,0,0.0,0,1,2,3,1,...,800.0,80.000000,115.000000,49.0,250.0,74.0,78.00000,104.0,80.0,0
2,Yosemite Ski & Snowboard Area,7800,600,0,0.0,0,0,0,1,3,...,88.0,174.873239,110.000000,84.0,300.0,47.0,47.00000,107.0,0.0,0
3,Boreal Mountain Resort,7700,500,0,0.0,0,1,1,3,1,...,380.0,200.000000,150.000000,54.0,400.0,49.0,64.16681,150.0,200.0,0
4,Dodge Ridge,8200,1600,0,0.0,0,0,1,2,5,...,862.0,174.873239,115.103943,69.0,350.0,78.0,78.00000,140.0,0.0,0


In [39]:
df.columns

Index(['Name', 'summit_elev', 'vertical_drop', 'trams', 'fastEight',
       'fastSixes', 'fastQuads', 'quad', 'triple', 'double', 'surface',
       'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'AdultWeekend', 'projectedDaysOpen',
       'NightSkiing_ac', 'clusters'],
      dtype='object')

## Standartize magnitude and numeric features

In [ ]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing as prp

# Declare an explanatory variable, called x,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
x = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter x 
scaler = prp.StandardScaler().fit(x)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter x 
x_scaled=scaler.transform(x) 

## Split into training and testing datasets

In [45]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.25, random_state=1)

In [46]:
# all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(x_train,y_train)

In [47]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(x_test)

In [48]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics.
from sklearn.metrics import explained_variance_score, mean_absolute_error

In [49]:
# You can plug y_test and y_pred into the functions to evaluate the model
evs = explained_variance_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [50]:
lm.intercept_

56.29586784469732

In [51]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
#pd.DataFrame(abs(lm.coef_), x.columns, columns=['Coefficient'])
coeff_df = pd.DataFrame(abs(lm.coef_), x.columns, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
AdultWeekday,10.603058
vertical_drop,4.202090
summit_elev,2.925350
TerrainParks,2.692877
quad,2.370920
daysOpenLastYear,2.361802
SkiableTerrain_ac,2.346417
projectedDaysOpen,2.248097
LongestRun_mi,1.805115
surface,1.371609


#### Model 3

**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

In [53]:
df.columns

Index(['Name', 'summit_elev', 'vertical_drop', 'trams', 'fastEight',
       'fastSixes', 'fastQuads', 'quad', 'triple', 'double', 'surface',
       'total_chairs', 'Runs', 'TerrainParks', 'LongestRun_mi',
       'SkiableTerrain_ac', 'Snow Making_ac', 'daysOpenLastYear', 'yearsOpen',
       'averageSnowfall', 'AdultWeekday', 'AdultWeekend', 'projectedDaysOpen',
       'NightSkiing_ac', 'clusters'],
      dtype='object')

In [54]:
df.drop(['summit_elev'], axis=1, inplace=True)

## Standartize magnitude and numeric features

In [55]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing as prp

# Declare an explanatory variable, called x,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
x = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter x 
scaler = prp.StandardScaler().fit(x)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter x 
x_scaled=scaler.transform(x) 

## Split into training and testing datasets¶

In [56]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.25, random_state=1)

In [57]:
# all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(x_train,y_train)

In [58]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(x_test)

In [59]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics.
from sklearn.metrics import explained_variance_score, mean_absolute_error

In [60]:
# You can plug y_test and y_pred into the functions to evaluate the model
evs = explained_variance_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [61]:
lm.intercept_

56.27615343499584

In [62]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
#pd.DataFrame(abs(lm.coef_), x.columns, columns=['Coefficient'])
coeff_df = pd.DataFrame(abs(lm.coef_), x.columns, columns=['Coefficient'])  
coeff_df.sort_values(by=['Coefficient'],ascending=False).head(10)

,Coefficient
AdultWeekday,10.748571
vertical_drop,4.297788
TerrainParks,2.728115
SkiableTerrain_ac,2.676886
projectedDaysOpen,2.415228
daysOpenLastYear,2.353467
quad,2.316499
LongestRun_mi,1.842096
clusters,1.369753
surface,1.191897


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0. |  |-|
| Model 2. | 0.| |'state'|
| Model 3. | 0. | |'state','summit_elev','base_elev'|

Model Selection: